In [1]:
!pip3 install scikit-learn
!pip3 install pandas

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd 
input_file = "cardio_train.csv"
df = pd.read_csv(input_file, header=0)

In [3]:
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder 

df_cat = df.select_dtypes(include='object')
encoder = OneHotEncoder(sparse=False, handle_unknown='error')
df_cat_encoded = encoder.fit_transform(df_cat)
categorical_columns = [f'{col}_{cat}' for i, col in enumerate(df_cat.columns) for cat in encoder.categories_[i]]
df_num = df.select_dtypes(exclude='object')
df_cat = pd.DataFrame(df_cat_encoded, columns=categorical_columns)
df = df_num.join(df_cat)

In [4]:
from sklearn.model_selection import train_test_split

Y = df[["cardio"]]
X = df.loc[:, df.columns != 'cardio']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()
y_train = y_train.ravel()
y_test = y_test.ravel()

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier


best_k_kf = 1
best_kf = 2
best_kf_value = 0
best_k_st = 1 
best_st = 2
best_st_value = 0

for k in range(1, 100, 5):
    model = KNeighborsClassifier(n_neighbors=k)

    for k_fold in range(2, 100, 20):
        kf = KFold(n_splits=k_fold)
        gs = GridSearchCV(model, cv=kf, param_grid={}, return_train_score=True)
        gs.fit(x_train, y_train)
        if gs.cv_results_['mean_test_score'][0] >= best_kf_value: 
            best_kf_value = gs.cv_results_['mean_test_score'][0]
            best_kf = k_fold
            best_k_kf = k 

        stf = StratifiedKFold(n_splits=k_fold)
        gs = GridSearchCV(model, cv=stf, param_grid={}, return_train_score=True)
        gs.fit(x_train, y_train)
        if gs.cv_results_['mean_test_score'][0] >= best_st_value: 
            best_st_value = gs.cv_results_['mean_test_score'][0]
            best_st = k_fold
            best_k_st = k 
        
print("Best k for k fold:", best_k_kf)
print("Best k fold:", best_kf)
print("Accuracy of best k fold:", best_kf_value)
print("Best k for stratified fold:", best_k_st)
print("Best stratified fold:", best_st)
print("Accuracy of best stratified fold:", best_st_value)

In [ ]:
from sklearn.metrics import accuracy_score

xs = []
ys_train = []
ys_test = []

for k in range(10, 60, 10):
    xs.append(k)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_train)
    ys_train.append(accuracy_score(y_train, y_pred))

    y_pred = model.predict(x_test)
    ys_test.append(accuracy_score(y_test, y_pred))

In [ ]:
from matplotlib import pyplot as plt 

plt.plot(xs, ys_train)
plt.show()

plt.plot(xs, ys_test)
plt.show()

We can pass weights parameter with value "